<a href="https://colab.research.google.com/github/ploffy89/Projet_Systemes_intelligents/blob/master/Projet_Syst%C3%A8mes_intelligents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorboardcolab

In [0]:
from __future__ import print_function

import keras
import math
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.callbacks import TensorBoard
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model


tbc=TensorBoardColab()

batch_size = 128
num_classes = 10
epochs = 40

datagen = ImageDataGenerator(horizontal_flip=True)

#def step_decay(epoch):
#	initial_lrate = 0.9
#	drop = 0.5
#	epochs_drop = 10.0
#	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
#	return lrate

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

img_rows, img_cols = 28, 28


(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

if K.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test= x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)  
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test= x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)  
  input_shape = (img_rows, img_cols, 1)  


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#model = Sequential()
#model.add(Conv2D(256, kernel_size=(3,3), activation='relu', input_shape=input_shape))
#model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(256, kernel_size=(1,1), activation='relu', input_shape=input_shape))
#model.add(Conv2D(32, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Flatten())

#model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
#model.add(Activation('relu'))
#model.add(Dense(64, activation='relu'))
#model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
#model.add(Activation('relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(num_classes, activation='softmax'))

model = load_model('denseNet.h5')
model.load_weights('denseNet_weights.h5')

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])

#lrate = LearningRateScheduler(step_decay)
callbacks_list = [#lrate,
                  reduce_lr,
                  TensorBoardColabCallback(tbc)]

model.summary()

#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data = (x_test, y_test))

model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size), validation_data = (x_test, y_test), steps_per_epoch=len(x_train)// batch_size,epochs=epochs, shuffle=True,callbacks=callbacks_list)

score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss :', score[0])
print('Test accuracy :', score[1])



Wait for 8 seconds...
TensorBoard link:
https://8b1aa7f1.ngrok.io
60000 train samples
10000 test samples
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 26, 26, 256)       2560      
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 24, 24, 64)        147520    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 12, 12, 256)       16640     
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 10, 10, 32)        73760     
_______________

# Save

In [0]:
model.save('denseNet.h5')
model.save_weights('denseNet_weights.h5')
#new_model.summary()
#new_model.get_weights()
#new_model.optimizer

# To JSON

In [0]:
json_string = model.to_json()
from keras.models import model_from_json
model_architecture = model_from_json(json_string)
model_architecture.summary()